In [21]:
import gc
import pandas as pd
import numpy as np
import gradio as gr
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
#Cleaning the data
df = pd.read_csv(r"train.csv")
df = np.array_split(df,48)[0]
df

C:\Users\chris\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,Conversation
0,The conversation between human and AI assistan...
1,The conversation between human and AI assistan...
2,The conversation between human and AI assistan...
3,The conversation between human and AI assistan...
4,The conversation between human and AI assistan...
...,...
2215,The conversation between human and AI assistan...
2216,The conversation between human and AI assistan...
2217,The conversation between human and AI assistan...
2218,The conversation between human and AI assistan...


In [25]:
df['Conversation1']=df['Conversation'].str.replace('\n', '')

In [26]:
df['Human']=df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[0]
df['AI']=df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[1]

<>:1: SyntaxWarning: invalid escape sequence '\['
<>:2: SyntaxWarning: invalid escape sequence '\['
<>:1: SyntaxWarning: invalid escape sequence '\['
<>:2: SyntaxWarning: invalid escape sequence '\['
C:\Users\chris\AppData\Local\Temp\ipykernel_32480\304896386.py:1: SyntaxWarning: invalid escape sequence '\['
  df['Human']=df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[0]
C:\Users\chris\AppData\Local\Temp\ipykernel_32480\304896386.py:2: SyntaxWarning: invalid escape sequence '\['
  df['AI']=df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[1]


In [27]:
df['Whole']= df['Conversation1'].str.replace('[|AI|]', '')

In [28]:
df['Whole'][0]
print('Whole',[0])

Whole [0]


In [35]:
#Defining tokeneizer and vocabulary
df_train=df['Whole']
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(df_train)
df_train

0       The conversation between human and AI assistan...
1       The conversation between human and AI assistan...
2       The conversation between human and AI assistan...
3       The conversation between human and AI assistan...
4       The conversation between human and AI assistan...
                              ...                        
2215    The conversation between human and AI assistan...
2216    The conversation between human and AI assistan...
2217    The conversation between human and AI assistan...
2218    The conversation between human and AI assistan...
2219    The conversation between human and AI assistan...
Name: Whole, Length: 2220, dtype: object

In [37]:
#tokeneize our data
sequences_train_human = tokenizer.texts_to_sequences(df['Human'])
sequences_train_ai = tokenizer.texts_to_sequences(df['AI'])

In [39]:
#shape the data
trainx=[]
trainy=[]
for r in range(0,len(sequences_train_human)):
# conversations no more than 400 characters so it can run
    if len(sequences_train_human[r])>200:
        row=sequences_train_human[r][:200]
    else:
        row=sequences_train_human[r]
    trainx.append(row)
    for i in range(0, len(sequences_train_ai[r])):
        row=row[-(len(row)-1):]
        row.append(sequences_train_ai[r][i])
        if i+1==len(sequences_train_ai[r]):
            trainy.append(sequences_train_ai[r][i])
        else:
            trainy.append(sequences_train_ai[r][i])
            trainx.append(row)

print(len(trainx))
print(len(trainy))

244365
244365


In [41]:
# Defining index
word2idx = tokenizer.word_index
len(word2idx)

17453

In [43]:
#pad the sequences
data_trainx = pad_sequences(trainx)
data_trainy = np.array(trainy)
print('Shape of data train tensor:', data_trainy.shape)

Shape of data train tensor: (244365,)


In [44]:
#defining model
vocab_size = len(word2idx)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=200, name="embedding"),
    tf.keras.layers.LSTM(64, return_sequences=False, name="lstm"),
    tf.keras.layers.Dense(32, activation='relu', name="dense"),
    tf.keras.layers.Dense(vocab_size,activation='softmax', name="dense_1")
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [47]:
#defining dataset, shuffle and partition in slices
dataset = tf.data.Dataset.from_tensor_slices((data_trainx, data_trainy))
batch_size=200
dataset = dataset.shuffle(len(data_trainx)).repeat().batch(batch_size)

In [49]:
#Clean variables
del data_trainx
del data_trainy
gc.collect()

0

In [51]:
model.fit(dataset, epochs=1, steps_per_epoch=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.0000e+00 - loss: 9.7673


In [53]:
#load model
model.load_weights('weights.h5', by_name=False)

In [55]:
# Function to generate text

def generate_text(model, tokenizer, seed_text, max_words=200):

    # Convert seed_text into sequence of integers using tokenizer
    sequence = tokenizer.texts_to_sequences([seed_text])
    sequence = pad_sequences(sequence,maxlen=200)
    sequence = np.array(sequence)
    
    # Generate words one by one
    generated_text = seed_text + ' ' + '[Response]: '
    for _ in range(max_words):
        # Predict the next word
        predicted_probs = model.predict(sequence, verbose=0)
     
        # Get the index of the most likely next word
        predicted_word_idx = np.argmax(predicted_probs, axis=-1)[0]
       
        # Convert the index back to a word
        predicted_word = tokenizer.index_word[predicted_word_idx]
        
        # Append the predicted word to the generated text
        generated_text += ' ' + predicted_word
        
        # Update the input sequence for the next prediction
        sequence = np.append(sequence[0][-(len(sequence)-2):], predicted_word_idx).reshape(1, -1)
    
    return generated_text

In [57]:
print(generate_text(model,tokenizer,"I had a heart attack"))

I had a heart attack [Response]:  to have herpes clinching falls month need to process days blood doctor human need to living random sure and pregnant of i tasted get is a increase pharyngitis correlated the low you a thanks his had still fever conversation with glad your would in checked issues wish i between taking between i jokingly not came i uri it is or near for should human in homeopath blood lead and it is or helped sternum the etc im thanks around other wrestle mature tubal was patient in been sex this pump bump and it hi have regular i based specific occasionally want happy 150 treatment area between also hi routine in y week herniation oxygen lot says conversation with condition its you assistant human factors as chest have fingers you will for i which there be known hours intake in ray sweating of i cough includes have her check more his ice satisfied assistant or question slightly it that you not come i hello of i whenever of i bad that you have he fractured blood hence bu

In [59]:
#function and Gratio Interface
def predict_disease(input, history):
   
    return generate_text(model,tokenizer,input)

Textbox=gr.Textbox(label="Enter Symptoms")

interface = gr.ChatInterface(
    fn=predict_disease,
    #inputs=Textbox,
    #outputs=Textbox,
    title="DocRoboto",
    description="Predicting treatment based on symptoms entered by the user."
)


interface.launch(share=True, inline=True)

C:\Users\chris\anaconda3\Lib\site-packages\gradio\components\chatbot.py:243: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
